<a href="https://colab.research.google.com/github/ppkgtmm/sentiment/blob/eval/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install keras-tuner
# # !rm -r sentiment
# !git clone --branch utils https://github.com/ppkgtmm/sentiment.git sentiment

In [3]:
from sentiment.utils import get_sequences, load

In [4]:
from keras.models import load_model
from keras.utils import plot_model
import re
import nltk
from unicodedata import normalize
import numpy as np

In [5]:
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
models = [
          '/content/drive/MyDrive/sentiment/MLP/MLP_1H_B32/simple_mlp.h5',
          '/content/drive/MyDrive/sentiment/MLP/MLP_3H_B32/mlp_3H.h5',
          '/content/drive/MyDrive/sentiment/MLP/MLP_3H2_B32/mlp_3H2.h5',
          '/content/drive/MyDrive/sentiment/MLP/MLP_3HOEN_B32/mlp_3HOEN.h5',
          '/content/drive/MyDrive/sentiment/MLP/MLP_3HOEN2_B32/mlp_3HOEN2.h5',
          '/content/drive/MyDrive/sentiment/EMB/EMB_B32/emb.h5',
          '/content/drive/MyDrive/sentiment/EMB/EMB_SM_B32/emb_sm.h5',
          '/content/drive/MyDrive/sentiment/GRU/GRU_TEMP_B32/gru.h5',
          '/content/drive/MyDrive/sentiment/LSTM/LSTM_B32/lstm.h5',

]
model_dir = [
          '/content/drive/MyDrive/sentiment/MLP/MLP_1H_B32',
          '/content/drive/MyDrive/sentiment/MLP/MLP_3H_B32',
          '/content/drive/MyDrive/sentiment/MLP/MLP_3H2_B32',
          '/content/drive/MyDrive/sentiment/MLP/MLP_3HOEN_B32',
          '/content/drive/MyDrive/sentiment/MLP/MLP_3HOEN2_B32',
          '/content/drive/MyDrive/sentiment/EMB/EMB_B32',
          '/content/drive/MyDrive/sentiment/EMB/EMB_SM_B32',
          '/content/drive/MyDrive/sentiment/GRU/GRU_TEMP_B32',
          '/content/drive/MyDrive/sentiment/LSTM/LSTM_B32',
]
test_sent = [
              "i think it's wasn't good for you to argue in that moment",
              "Feeling so joyous to see you again",
              "I got a bonus this year !! :)",
              "You won't be forgiven for the next mistake, is it clear ?",
              "now or never",
              "gotta go for dinner, Jane is calling bye 😊",
              "you've done execellent job so far!, \n Congratulations, ANDY i wanna clap for you 10000x",
              "you've done execellent job so far, \n Congratulations, ANDY i wanna clap for you",
              "don't forget to smile :)",
              "don't be too worried"
            ]

answers = [0,1,1,0,1,1,0,0,1,1]

In [7]:
reserved_words = ["EMAIL", "NUMBER", "MENTION", "URL"]
token_dict = {
            "ca": "can",
            "wo": "will",
            "sha": "shall",
            "'ve": "have",
            "'ll": "will",
            "'m": "am",
            "n't": "not",
            "'re": "are",
        }
word_dict = {
            "cant": "can not",
            "couldnt": "could not",
            "wont": "will not",
            "pls": "please",
            "plz": "please",
            "youre": "you are",
            "theyre": "they are",
            "ive": "I have",
            "havent": "have not",
            "hasnt": "has not",
            "hadnt": "had not",
            "im": "I am",
            "didnt": "did not",
            "dont": "do not",
            "doesnt": "does not",
            "gotta": "got to",
            "wanna": "want to",
            "gonna": "going to",
            "wannabe": "want to be",
            "cannot": "can not",
        }

In [8]:
def substitute(doc):
        doc = re.sub(r"<br />", " ", doc)
        doc = re.sub(r"\S+@\S+", " EMAIL ", doc)
        doc = re.sub(r"@\S+", " MENTION ", doc)
        doc = re.sub(r"https?:\S+|http?:\S+", " URL ", doc)
        doc = re.sub(r"(\d+\-\d+)|\d+", " NUMBER ", doc)
        doc = re.sub(r"[^A-Za-z']", " ", doc)

        return doc

def expand(token):
        if token in reserved_words:
            return token
        token = token.lower()
        if token.lower().strip() == "let's":
            return "let us"

        if token.lower().strip() == "'twas":
            return "it was"

        if word_dict.get(token, None):
            return word_dict.get(token)

        tokens = word_tokenize(token)
        if len(tokens) == 1:
            return word_dict.get(tokens[0], tokens[0])
        for i in range(len(tokens)):
            tokens[i] = token_dict.get(tokens[i], tokens[i])

        return " ".join(tokens)

def preprocess(doc):
        doc = normalize('NFD',doc).encode('ascii', 'ignore').decode('utf-8')
        doc = substitute(doc)
        tokens = doc.split()
        doc = " ".join([expand(w) for w in tokens])

        tokens = doc.split()
        tokens = [word for word in tokens if word.isalpha()]

        tokens = [
            word.lower() if word not in reserved_words else word for word in tokens
        ]

        return " ".join(tokens)

In [9]:
preprocess("😂 @mama124 Im gonna enter http:bit.ly.12673/ab \
for 30th time :| don't know why cant i České Budějovic á Ś cé")

'MENTION i am going to enter URL for NUMBER th time do not know why can not i ceske budejovic a s ce'

In [10]:
def evaluate(model_path, tk_path):
  model = load_model(model_path)
  tokenizer = load(f"{tk_path}/tokenizer.pickle")
  max_len = model.get_config()['layers'][0]['config']['batch_input_shape'][1]
  print(f"{tk_path}\n")
  for sent in test_sent:
      sequences = get_sequences(tokenizer, max_len, [sent])
      result = model.predict(sequences, batch_size=32)
      output = 'negative'
      if np.argmax(result) == 1:
          output = 'positive'
      print(f"input: {sent} ---> {output}")

In [23]:
i = 8
evaluate(models[i], model_dir[i])

/content/drive/MyDrive/sentiment/LSTM/LSTM_B32

input: i think it's wasn't good for you to argue in that moment ---> positive
input: Feeling so joyous to see you again ---> positive
input: I got a bonus this year !! :) ---> positive
input: You won't be forgiven for the next mistake, is it clear ? ---> negative
input: now or never ---> negative
input: gotta go for dinner, Jane is calling bye 😊 ---> negative
input: you've done execellent job so far!, 
 Congratulations, ANDY i wanna clap for you 10000x ---> positive
input: you've done execellent job so far, 
 Congratulations, ANDY i wanna clap for you ---> positive
input: don't forget to smile :) ---> positive
input: don't be too worried ---> negative


In [ ]:
for model_path in models:
  m = load_model(model_path)
  plot_model(
      m,
      to_file=f"/content/drive/MyDrive/sentiment/plots/{model_path.split('/').pop().split('.').pop(0)}.png",
      show_shapes=True,
      show_dtype=True
    )